In [2]:
import os
import pandas as pd
import numpy as np
from IPython.display import display
import h5py
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.cluster import KMeans
import logging
import math
import numpy as np
import torch
from torch.nn import ModuleList
from functools import partial
from typing import List, Tuple, Dict, Optional, Any
from torchvision.transforms import Compose, RandomApply
from torchvision.transforms import functional as F
from torchvision.transforms.transforms import _setup_angle, _check_sequence_input
from torch import Tensor
from collections import defaultdict, deque
from pathlib import Path
from torch import nn
from PIL import ImageFilter, ImageOps, Image, ImageDraw

import os
# 设定实验结果的目录 (请修改为你的实际路径)
result_root_dir = '/home/yuhaowang/project/FMBC/downstream/finetune/outputs'
for dir in os.listdir(result_root_dir):
    print(dir)
    result_dir = os.path.join(result_root_dir, dir)
# 需要展示的评估指标
    evaluation_metrics = ['val_bacc', 'val_weighted_f1', 'val_macro_auroc']

    # 你希望的模型顺序（从上到下）
    desired_order = [
        "UNI", "CONCH", "Virchow","Gigapath_Tile",'Gigapath',
        "CHIEF_Tile","TITAN","FMBC"  # 请修改为你的模型名称
    ]

    all_results = []

    # 遍历目录中的所有模型文件夹
    for model_name in os.listdir(result_dir):
    # model_path = os.path.join(result_dir, model_name, "summary.csv")
        for tuning_method in os.listdir(os.path.join(result_dir, model_name)):
            model_sumary_path = os.path.join(result_dir, model_name, tuning_method, "summary.csv")
            if 'ABMIL' in model_sumary_path:
                continue
            if os.path.isfile(model_sumary_path):
                df = pd.read_csv(model_sumary_path)

                # 计算均值和标准差
                summary_stats = {"Model": model_name+'_'+tuning_method}
                for metric in evaluation_metrics:
                    if metric in df.columns:
                        mean_val = np.mean(df[metric])
                        std_val = np.std(df[metric], ddof=1)  # 样本标准差
                        summary_stats[metric] = f"{mean_val:.3f}±{std_val:.4f}"

                # 添加到列表
                all_results.append(summary_stats)

    # 转换为 DataFrame
    final_result_df = pd.DataFrame(all_results)

    # 按照提供的模型顺序排序
    # final_result_df['sort_order'] = final_result_df['Model'].apply(lambda x: desired_order.index(x) if x in d
    #                                                                esired_order else len(desired_order))
    #delete the model not in desired_order
    #final_result_df = final_result_df[final_result_df['sort_order']!=len(desired_order)]
    #final_result_df = final_result_df.sort_values(by='sort_order').drop(columns=['sort_order'])
    final_result_df.style.hide(axis="index")
    # 在 Jupyter Notebook 中美观显示
    display(final_result_df)




AIDPATH_GRADE


""


BRACS_FINE


""


SLNBREAST_SUBTYPE


...

""


BCNB_ALN


""


AIDPATH_IDC


""


In [3]:
import pandas as pd 
data = pd.read_csv('/home/yuhaowang/project/FMBC/downstream/finetune/dataset_csv/subtype/BCNB_ALN.csv')
data['label'].value_counts()

label
N0         655
N+(1-2)    210
N+(>2)     193
Name: count, dtype: int64

In [ ]:
import numpy as np
import scipy.spatial.distance as distance
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import torch

def get_few_shot_samples(X, y, k):
    """Select k samples per class for few-shot learning."""
    unique_classes = np.unique(y)
    few_shot_X, few_shot_y = [], []
    
    for c in unique_classes:
        class_samples = X[y == c]
        selected_samples = class_samples[:min(k, len(class_samples))]
        few_shot_X.append(selected_samples)
        few_shot_y.append(np.full(len(selected_samples), c))
    
    return np.vstack(few_shot_X), np.hstack(few_shot_y)

class SimpleShot:
    def __init__(self):
        self.class_prototypes = None
        self.classes = None
    
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.class_prototypes = {
            c: np.mean(X[y == c], axis=0) for c in self.classes
        }
    
    def predict(self, X):
        preds = []
        for x in X:
            distances = {c: np.linalg.norm(x - proto) for c, proto in self.class_prototypes.items()}
            preds.append(min(distances, key=distances.get))
        return np.array(preds)

class Retrieval:
    def __init__(self, database, labels):
        self.database = database
        self.labels = labels
    
    def retrieve(self, query, k=5):
        dists = np.linalg.norm(self.database - query, axis=1)
        indices = np.argsort(dists)[:k]
        return self.labels[indices]
    
    def evaluate(self, queries, query_labels, k_vals=[1, 3, 5]):
        acc_at_k = {k: 0 for k in k_vals}
        mvacc_at_5 = 0
        
        for i, query in enumerate(queries):
            retrieved_labels = self.retrieve(query, k=max(k_vals))
            for k in k_vals:
                if query_labels[i] in retrieved_labels[:k]:
                    acc_at_k[k] += 1
            if np.bincount(retrieved_labels[:5]).argmax() == query_labels[i]:
                mvacc_at_5 += 1
        
        num_queries = len(queries)
        return {k: acc_at_k[k] / num_queries for k in k_vals}, mvacc_at_5 / num_queries

def few_shot_experiment(k, X_train, y_train, X_test, y_test):
    """Run few-shot learning experiments for a given k using real dataset."""
    X_few_shot, y_few_shot = get_few_shot_samples(X_train, y_train, k)
    
    # SimpleShot Evaluation
    simpleshot = SimpleShot()
    simpleshot.fit(X_few_shot, y_few_shot)
    y_pred = simpleshot.predict(X_test)
    simpleshot_acc = accuracy_score(y_test, y_pred)
    
    # 20-Nearest Neighbors Evaluation
    knn = KNeighborsClassifier(n_neighbors=20)
    knn.fit(X_few_shot, y_few_shot)
    y_pred_knn = knn.predict(X_test)
    knn_acc = accuracy_score(y_test, y_pred_knn)
    
    # Linear Probing Evaluation
    linear_probe = LogisticRegression(max_iter=1000, C=1.0)
    linear_probe.fit(X_few_shot, y_few_shot)
    y_pred_linear = linear_probe.predict(X_test)
    linear_acc = accuracy_score(y_test, y_pred_linear)
    
    # Retrieval Evaluation
    retrieval = Retrieval(X_few_shot, y_few_shot)
    acc_at_k, mvacc_at_5 = retrieval.evaluate(X_test, y_test)
    
    return {
        "SimpleShot Accuracy": simpleshot_acc,
        "20-NN Accuracy": knn_acc,
        "Linear Probing Accuracy": linear_acc,
        "Retrieval Acc@K": acc_at_k,
        "MVAcc@5": mvacc_at_5
    }


SimpleShot Accuracy: 0.8200
20-NN Accuracy: 0.8200
Linear Probing Accuracy: 0.8600
Retrieval Acc@K: {1: 0.78, 3: 0.96, 5: 0.98}, MVAcc@5: 0.8600


In [6]:
import pandas as pd 
data = pd.read_csv('/home/yuhaowang/project/FMBC/downstream/finetune/outputs/BRACS_COARSE/CONCH/LR/summary.csv')
data

,val_loss,val_bacc,val_acc,val_macro_auroc,val_macro_auprc,val_weighted_f1,val_qwk,val_0_auroc,val_1_auroc,val_2_auroc,...,test_macro_auroc,test_macro_auprc,test_weighted_f1,test_qwk,test_0_auroc,test_1_auroc,test_2_auroc,test_0_auprc,test_1_auprc,test_2_auprc
0,0.834959,0.475862,0.654545,0.750391,0.642540,0.516206,0.406843,0.753316,0.714286,0.783571,...,0.702350,0.540863,0.363388,0.381250,0.756065,0.603333,0.747652,0.763799,0.223429,0.635361
1,0.938483,0.514254,0.618182,0.731070,0.594004,0.489013,0.483366,0.748992,0.611434,0.832785,...,0.720668,0.584372,0.459933,0.548828,0.742608,0.585514,0.833882,0.666225,0.264163,0.822727
2,0.899061,0.483583,0.627273,0.738174,0.591267,0.507941,0.420513,0.757129,0.722328,0.735065,...,0.731122,0.576863,0.474645,0.421699,0.749834,0.716951,0.726582,0.702118,0.329745,0.698724
3,0.909625,0.450926,0.563636,0.754923,0.605143,0.427175,0.317882,0.790940,0.655281,0.818548,...,0.755381,0.604385,0.417920,0.299271,0.789231,0.654333,0.822581,0.727746,0.276788,0.808622
4,0.866451,0.524402,0.681818,0.785006,0.639934,0.537006,0.522155,0.768484,0.762778,0.823755,...,0.777716,0.626934,0.454448,0.491189,0.766310,0.745000,0.821839,0.803038,0.320038,0.757726


In [1]:
import os
import pandas as pd
import numpy as np
from IPython.display import display
import h5py
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.cluster import KMeans
import logging
import math
import numpy as np
import torch
from torch.nn import ModuleList
from functools import partial
from typing import List, Tuple, Dict, Optional, Any
from torchvision.transforms import Compose, RandomApply
from torchvision.transforms import functional as F
from torchvision.transforms.transforms import _setup_angle, _check_sequence_input
from torch import Tensor
from collections import defaultdict, deque
from pathlib import Path
from torch import nn
from PIL import ImageFilter, ImageOps, Image, ImageDraw

import os

result_root_dir = '/home/yuhaowang/project/FMBC/downstream/finetune/outputs'
for dir in os.listdir(result_root_dir):
    print(dir)
    result_dir = os.path.join(result_root_dir, dir)

    evaluation_metrics = ['val_bacc', 'val_weighted_f1', 'val_macro_auroc']


    desired_order = [
        "UNI", "CONCH", "Virchow","Gigapath_Tile",'Gigapath',
        "CHIEF_Tile","TITAN","FMBC"  # 请修改为你的模型名称
    ]

    all_results = []

    # 遍历目录中的所有模型文件夹
    for model_name in os.listdir(result_dir):
    # model_path = os.path.join(result_dir, model_name, "summary.csv")
        for tuning_method in os.listdir(os.path.join(result_dir, model_name)):
            for lr_rate in os.listdir(os.path.join(result_dir, model_name, tuning_method)):
                model_sumary_path = os.path.join(result_dir, model_name, tuning_method,lr_rate, "summary.csv")
                if 'ABMIL' in model_sumary_path:
                    continue
                if os.path.isfile(model_sumary_path):
                    df = pd.read_csv(model_sumary_path)

                    # 计算均值和标准差
                    summary_stats = {"Model": model_name+'_'+tuning_method+'_'+lr_rate}
                    for metric in evaluation_metrics:
                        if metric in df.columns:
                            mean_val = np.mean(df[metric])
                            std_val = np.std(df[metric], ddof=1)  # 样本标准差
                            summary_stats[metric] = f"{mean_val:.3f}±{std_val:.4f}"

                    # 添加到列表
                    all_results.append(summary_stats)


    final_result_df = pd.DataFrame(all_results)


    # final_result_df['sort_order'] = final_result_df['Model'].apply(lambda x: desired_order.index(x) if x in d
    #                                                                esired_order else len(desired_order))
    #delete the model not in desired_order
    #final_result_df = final_result_df[final_result_df['sort_order']!=len(desired_order)]
    #final_result_df = final_result_df.sort_values(by='sort_order').drop(columns=['sort_order'])
    final_result_df.style.hide(axis="index")
    # 在 Jupyter Notebook 中美观显示
    display(final_result_df)




AIDPATH_GRADE


,Model,val_bacc,val_weighted_f1,val_macro_auroc
0,TITAN_LR_0.001,0.784±0.0677,0.804±0.0482,0.864±0.1187
1,TITAN_LR_0.1,0.823±0.0669,0.844±0.0455,0.894±0.0968
2,TITAN_LR_0.01,0.834±0.0665,0.857±0.0453,0.890±0.1185
3,TITAN_LR_0.0001,0.683±0.0656,0.723±0.0620,0.832±0.1120
4,CHIEF_LR_0.001,0.751±0.0727,0.781±0.0545,0.834±0.1102
5,CHIEF_LR_0.1,0.843±0.1019,0.860±0.0773,0.911±0.1223
6,CHIEF_LR_0.01,0.825±0.0855,0.845±0.0615,0.893±0.1262
7,CHIEF_LR_0.0001,0.500±0.0000,0.488±0.0376,0.773±0.0973
8,UNI_LR_0.001,0.822±0.0831,0.843±0.0598,0.898±0.0942
9,UNI_LR_0.1,0.899±0.0928,0.912±0.0695,0.925±0.0962


BCNB_ALN


""


AIDPATH_IDC


,Model,val_bacc,val_weighted_f1,val_macro_auroc
0,Gigapath_tile_LR_0.001,0.834±0.0911,0.898±0.1248,0.934±0.0949
1,Gigapath_tile_LR_0.1,0.900±0.0779,0.913±0.1196,0.949±0.1011
2,Gigapath_tile_LR_0.01,0.903±0.0733,0.924±0.1174,0.944±0.1129
3,Gigapath_tile_LR_0.0001,0.500±0.0000,0.781±0.0590,0.845±0.0834
